In [1]:
#!/usr/bin/python3
'''
Author: Zhao-Shun Zheng
Date: 2021/6/19
function: Publish datas from sensors by MQTT
email: e14051148@gs.ncku.edu.tw
'''
import paho.mqtt.client as mqtt
import random
import json  
import datetime 
import time
import pandas as pd
import cv2
import numpy as np

HOST = "192.168.0.147"
PORT = 1883  

client = mqtt.Client()

client.connect(HOST, PORT, 60)

count_5 = 0
count_2 = 0
count_6 = 0
count_1 = 0

data_group5 = pd.read_csv('./Data/Group5_Start_1.csv')
data_group2 = pd.read_csv('./Data/Group2_Tracking_2.csv')
data_group6 = pd.read_csv('./Data/Group6_defect_3.csv')
data_group1 = pd.read_csv('./Data/Group1_Shipment_4.csv')

ISOTIMEFORMAT = '%Y/%m/%d %H:%M:%S'
print(datetime.datetime.now().strftime(ISOTIMEFORMAT))

2021/06/18 17:37:10


In [2]:
for i in range(0, len(data_group2)):

# -----------------Start------------------- #
    if str(data_group2["位置"][count_2]) == '0':
        payload5 = (  str(data_group5["訂單編號"][count_5]) + ',' 
                   + str(data_group5["工件編號"][count_5]) + ',' 
                   + str(data_group5["執行時間"][count_5]) + ',' 
                   + str(data_group5["物件種類"][count_5]))
        client.publish("IOT/Group5", bytes(payload5, encoding = "utf-8"))
        count_5 = count_5 + 1
# -----------------Start------------------- #

# -----------------detect defect------------------- #
    if str(data_group2["位置"][count_2]) == '150':
        path = 'Image/' + str(data_group6["影像"][count_6])
        img = cv2.imread(path)
        image = img.tolist()
        payload6 = (  str(data_group6["訂單編號"][count_6]) + ',' 
                   + str(data_group6["工件編號"][count_6]) + ',' 
                   + str(data_group6["瑕疵檢測"][count_6]))
        message =  { 'payload' : payload6, 'image' : image } 
        client.publish("IOT/Group6", json.dumps(message))
        count_6 = count_6 + 1
# -----------------detect defect------------------- #

# -----------------Finish------------------- #
    if str(data_group2["位置"][count_2]) == '200':
        payload1 = (  str(data_group1["訂單編號"][count_1]) + ',' 
               + str(data_group1["工件編號"][count_1]) + ',' 
               + str(data_group1["夾取時間"][count_1]) + ',' 
               + str(data_group1["出貨時間"][count_1]) + ',' 
               + str(data_group1["出貨地點"][count_1]))
        client.publish("IOT/Group1", bytes(payload1, encoding = "utf-8"))
        count_1 = count_1 + 1
# -----------------Finish------------------- #

# -----------------Track position------------------- # 
    payload2 = (  str(data_group2["訂單編號"][count_2]) + ',' 
               + str(data_group2["工件編號"][count_2]) + ',' 
               + str(data_group2["位置"][count_2])) 
    client.publish("IOT/Group2", bytes(payload2, encoding = "utf-8"))
    count_2 = count_2 + 1
# -----------------Track position------------------- #
    
    time.sleep(1)